In [38]:
from langchain_experimental.agents import create_csv_agent
from dotenv import load_dotenv
from langchain_openai import AzureChatOpenAI
import os
load_dotenv()
import pandas as pd
from langchain_community.document_loaders import JSONLoader

In [2]:
llm = AzureChatOpenAI(openai_api_version=os.environ.get("AZURE_OPENAI_VERSION", "2023-07-01-preview"),
        azure_deployment=os.environ.get("AZURE_OPENAI_DEPLOYMENT", "gpt4chat"),
        azure_endpoint=os.environ.get("AZURE_OPENAI_ENDPOINT", "https://gpt-4-trails.openai.azure.com/"),
        api_key=os.environ.get("AZURE_OPENAI_KEY"))

In [5]:

def metadata_func(record: str, metadata: dict) -> dict:
    lines = record.split('\n')
    locality_line = lines[10]
    price_range_line = lines[12]
    locality = locality_line.split(': ')[1]
    price_range = price_range_line.split(': ')[1]
    metadata["location"] = locality
    metadata["price_range"] = price_range

    return metadata

# Instantiate the JSONLoader with the metadata_func
jq_schema = '.parser[] | to_entries | map("\(.key): \(.value)") | join("\n")'
loader = JSONLoader(
    jq_schema=jq_schema,
    file_path='data.json',
    metadata_func=metadata_func,
)

# Load the JSON file and extract metadata
documents = loader.load()

# Print the metadata of the first document
print(documents[0].metadata)

{'source': '/Users/siddartha/Desktop/github/Traversaal_ai/data.json', 'seq_num': 1, 'location': 'Istanbul', 'price_range': 'low'}


In [14]:
print(type(documents[0]))

<class 'langchain_core.documents.base.Document'>


In [21]:
text = [i.page_content for i in documents]

In [34]:
from langchain_openai import AzureOpenAIEmbeddings
embeddings = AzureOpenAIEmbeddings(
    openai_api_version=os.environ.get("AZURE_OPENAI_VERSION", "2023-07-01-preview"),
    azure_deployment=os.environ.get("embeddings","embed"),
    azure_endpoint=os.environ.get("AZURE_OPENAI_ENDPOINT", "https://gpt-4-trails.openai.azure.com/"),
    api_key=os.environ.get("AZURE_OPENAI_KEY")
)

In [57]:
from langchain_openai import OpenAIEmbeddings
import os

In [58]:
from langchain.vectorstores import FAISS
def get_vectorstore(text_chunks):
    embeddings = OpenAIEmbeddings()
    vectorstore = FAISS.from_documents(documents=text_chunks, embedding=embeddings)
    return vectorstore

In [59]:
vector = get_vectorstore(documents)

In [72]:
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
template = """

context:- I have low budget what is the best hotel in Instanbul?
anser:- The other hotels in instanbul are costly and are not in your budget. so the best hotel in instanbul for you is hotel is xyz.


Don’t give information not mentioned in the CONTEXT INFORMATION. 
The system should take into account various factors such as location, amenities, user reviews, and other relevant criteria to 
generate informative and personalized explanations.
{context}
Question: {question}
Answer:"""
 
prompt = PromptTemplate(template=template, input_variables=["context", "question"])
chain_type_kwargs = {"prompt": prompt}
chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vector.as_retriever(),
    chain_type_kwargs=chain_type_kwargs,
)

In [73]:
response = chain.run("I have low budget what is the best hotel in Instanbul?")
print(response)

Considering your budget and the hotel reviews, the best hotel for you in Istanbul would be the Mula Hotel. It is reasonably priced and has received excellent reviews from guests, with a rating value of 5 from 197 reviews. It is located in the heart of the city, close to popular historical attractions like the Blue Mosque, Hagia Sophia, and the Topkapi Palace. Guests have praised the hotel's intimate atmosphere, modern architecture, serene ambiance, and the breathtaking views of the Sea of Marmara. [Here is the link to the hotel.](https://www.tripadvisor.com/Hotel_Review-g293974-d24067960-Reviews-or30-Mula_Hotel-Istanbul.html)


In [71]:
response = chain.run("Give me a hotel that is near a water body?")
print(response)

The Wyndham Grand Istanbul Kalamis Marina Hotel is near a water body. Located in Istanbul, Turkiye, this hotel is situated on the Fener Kalamis Caddesi. The hotel has a high rating of 4.5 and offers amenities such as anti-allergic and disabled rooms.


In [48]:
from qdrant_client.http.models import Batch
import openai

In [50]:
def encode_text(text):
    response = openai.Embedding.create(inputs=text)
    return response['embedding']

In [54]:
!pip install openai

In [55]:
from openai.api import OpenAIApi
# Define a query vector
query_vector = encode_text("query text")

# Retrieve relevant documents based on the query vector
relevant_documents = qdrant.get_relevant_documents(query_vector)

# Use OpenAI's language model to generate answers based on the retrieved documents
# Example: Use the retrieved documents as context for a language model completion
response = OpenAIApi().complete(
    model="text-davinci-003",
    documents=relevant_documents,
    prompt="Generate an answer based on the retrieved documents."
)

print(response.choices[0].text)

ModuleNotFoundError: No module named 'openai.api'